## Imports

In [1]:
import yfinance as yf
import plotly.graph_objs as go
import numpy as np

## Loading Data

In [2]:
stock = 'TSLA'
start_date = '2020-01-01'
end_date = '2021-01-01'

ticker = yf.Ticker(stock)
df = ticker.history(interval='1d',
                    start=start_date,
                    end=end_date)
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02 00:00:00-05:00,28.299999,28.713333,28.114000,28.684000,142981500,0.0,0.0
2020-01-03 00:00:00-05:00,29.366667,30.266666,29.128000,29.534000,266677500,0.0,0.0
2020-01-06 00:00:00-05:00,29.364668,30.104000,29.333332,30.102667,151995000,0.0,0.0
2020-01-07 00:00:00-05:00,30.760000,31.441999,30.224001,31.270666,268231500,0.0,0.0
2020-01-08 00:00:00-05:00,31.580000,33.232666,31.215334,32.809334,467164500,0.0,0.0


## Extraindo médias móveis

In [3]:
df['mv_7d_1.5_std'] = df['Close'].rolling(7).mean() + df['Close'].rolling(7).std() * 1.5
df['mv_50d'] = df['Close'].rolling(50).mean()

## Visualizando dados

In [4]:
def plotCandleStick(df_, name='ticket', lines=[]):
  tracel = {
      'x': df_.index,
      'open': df_.Open,
      'close': df_.Close,
      'high': df_.High,
      'low': df_.Low,
      'type': 'candlestick',
      'name': name,
      'showlegend':False
  }

  data = [tracel]
  layout = go.Layout()

  fig = go.Figure(data=data , layout=layout)

  if len(lines) > 0:
    for c in lines:
      fig.add_trace(
          go.Scatter(x = list(df_.index),
                     y = df_[c],
                     mode = 'lines',
                     name = c))

  return fig

In [5]:
plotCandleStick(df, name=stock, lines=['mv_7d_1.5_std', 'mv_50d'])

In [6]:
df.reset_index(inplace=True)

## Criando coluna operação

In [7]:
df['sinal_operacao'] = np.NaN

idx_venda = df[df['Close'] > df['mv_7d_1.5_std']].index 
idx_compra = df[df['Close'] < df['mv_50d']].index 

df.loc[idx_venda, 'sinal_operacao'] = 'venda'
df.loc[idx_compra, 'sinal_operacao'] = 'compra'

## Dropping sinal de operação NaN

In [8]:
df.dropna(subset = ['sinal_operacao'], inplace=True)

## Análisando os dados

In [9]:
def clean_consecutive_duplicates(df_, subset=None, keep='first', inplace=False):
  if keep == 'first':
    n=1
  else:
    n=-1

  if subset is None:
    filter_ = (df_.shift(n) != df_).any(axis=1)
  else:
    filter_ = (df_[subset].shift(n) != df_[subset]).any(axis=1)

  if inplace:
    df_.drop( index=df_[-filter_].index, inplace=True)
  else:
    return df_.loc[filter_]  

In [10]:
clean_consecutive_duplicates(df, keep='first', subset=['sinal_operacao'], inplace=True)

## Verificando a rentabilidade

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,mv_7d_1.5_std,mv_50d,sinal_operacao
0,2020-01-13 00:00:00-05:00,32.900002,35.042000,32.799999,34.990665,397764000,0.0,0.0,34.520445,NaN,venda
1,2020-03-13 00:00:00-04:00,39.666668,40.504665,33.466667,36.441334,339604500,0.0,0.0,48.808023,43.886280,compra
2,2020-04-13 00:00:00-04:00,39.344002,43.466667,38.702000,43.396667,337131000,0.0,0.0,42.345705,43.137080,venda
3,2020-10-28 00:00:00-04:00,138.826660,139.533340,135.333328,135.339996,76354200,0.0,0.0,143.395267,140.640666,compra
4,2020-11-05 00:00:00-05:00,142.766663,146.666672,141.333328,146.029999,85243500,0.0,0.0,145.816494,141.028266,venda


In [13]:
df['rentab'] = df['Close'] / df['Close'].shift() * 100 -100

## Simulando operações

In [14]:
valor_investido = 10000
df['Saldo'] = np.NaN

idx_venda = df[df['sinal_operacao'] == 'venda'].index
idx_compra = df[df['sinal_operacao'] == 'compra'].index

df.loc[idx_venda, 'Saldo'] = (valor_investido * (100 + df['rentab'])) / 100 - valor_investido

In [15]:
df[['Date','Close','rentab','Saldo','mv_7d_1.5_std','mv_50d']]

,Date,Close,rentab,Saldo,mv_7d_1.5_std,mv_50d
0,2020-01-13 00:00:00-05:00,34.990665,NaN,NaN,34.520445,NaN
1,2020-03-13 00:00:00-04:00,36.441334,4.145872,NaN,48.808023,43.886280
2,2020-04-13 00:00:00-04:00,43.396667,19.086386,1908.638623,42.345705,43.137080
3,2020-10-28 00:00:00-04:00,135.339996,211.867257,NaN,143.395267,140.640666
4,2020-11-05 00:00:00-05:00,146.029999,7.898628,789.862770,145.816494,141.028266
5,2020-11-09 00:00:00-05:00,140.419998,-3.841677,NaN,147.844372,140.766799
6,2020-11-17 00:00:00-05:00,147.203339,4.830751,483.075099,144.926837,140.241332


In [16]:
df['Saldo'].sum()

3181.5764919695375